# Movimiento de órdenes de FarmAhorra de Silver a Gold

## 1. Contexto Unity-Catalog

In [0]:
%sql
USE CATALOG farma;
CREATE SCHEMA IF NOT EXISTS gold;
USE SCHEMA gold;

## 2. Creación de Tabla base con KPI diarios (calculados por día y farmacia)

In [0]:
%sql
CREATE OR REPLACE TABLE farma.gold.daily_kpis_by_pharmacy_base AS
SELECT
  order_date,
  id_farmacia,
  COUNT(*) AS orders_cnt,
  COUNT(DISTINCT client_id) AS customers_cnt,
  SUM(COALESCE(subtotal, 0.0)) AS gross_sales, -- ventas sin descuento
  SUM(COALESCE(discount, 0.0)) AS discount_amt,
  SUM(COALESCE(total, 0.0)) AS net_sales, -- ventas con descuento aplicado
  CASE WHEN SUM(COALESCE(subtotal, 0.0)) > 0
       THEN SUM(COALESCE(discount, 0.0)) / SUM(COALESCE(subtotal, 0.0))
       ELSE 0.0 END AS discount_rate, -- tasa de descuento
  CASE WHEN COUNT(*) > 0
       THEN SUM(COALESCE(total, 0.0)) / COUNT(*)
       ELSE 0.0 END AS avg_order_value -- costo promedio de orden
FROM farma.silver.orders
GROUP BY order_date, id_farmacia;

num_affected_rows,num_inserted_rows


## 3. Creación de vista con KPIs acumulados diarios por farmacia

In [0]:
%sql
CREATE OR REPLACE VIEW farma.gold.daily_kpis_by_pharmacy_cum AS
SELECT
  order_date,
  id_farmacia,
  orders_cnt,
  customers_cnt,
  gross_sales,
  discount_amt,
  net_sales,
  discount_rate,
  avg_order_value,
  SUM(orders_cnt) OVER (PARTITION BY id_farmacia ORDER BY order_date) AS cum_orders_cnt,
  SUM(net_sales) OVER (PARTITION BY id_farmacia ORDER BY order_date) AS cum_net_sales,
  SUM(discount_amt) OVER (PARTITION BY id_farmacia ORDER BY order_date) AS cum_discount_amt
FROM farma.gold.daily_kpis_by_pharmacy_base;


## 4. Creación de tabla para cálculo de métricas por cliente

In [0]:
%sql
CREATE OR REPLACE TABLE farma.gold.client_lifetime AS
SELECT
  client_id,
  MIN(order_date) AS first_order_date,
  MAX(order_date) AS last_order_date,
  COUNT(*) AS orders_cnt,
  SUM(COALESCE(total, 0.0))  AS net_sales,
  CASE WHEN COUNT(*) > 0
       THEN SUM(COALESCE(total, 0.0)) / COUNT(*)
       ELSE 0.0 END  AS avg_order_value
FROM farma.silver.orders
GROUP BY client_id;


num_affected_rows,num_inserted_rows


## 5. Creación de vista para enlistar ventas y ordenes por clientes

In [0]:
%sql
CREATE OR REPLACE VIEW farma.gold.top_clients_by_orders AS
SELECT
  client_id,
  first_order_date,
  last_order_date,
  orders_cnt,
  net_sales,
  avg_order_value,
  DENSE_RANK() OVER (ORDER BY orders_cnt DESC, net_sales DESC, client_id) AS rnk_by_orders
FROM farma.gold.client_lifetime;

## 6. Creación de tabla con productos más vendidos por farmacia

In [0]:
%sql
-- Tabla base: ventas y unidades por producto x farmacia
CREATE OR REPLACE TABLE farma.gold.product_lifetime_by_pharmacy AS
SELECT
  id_farmacia,
  package_ndc_11,
  descripcion AS product_name,
  COUNT(*)                             AS orders_cnt,
  SUM(COALESCE(quantity, 0))           AS units_sold,
  SUM(COALESCE(total, 0.0))            AS sales_usd
FROM farma.silver.orders
WHERE package_ndc_11 IS NOT NULL
GROUP BY id_farmacia, package_ndc_11, descripcion;

num_affected_rows,num_inserted_rows


## 7. Creación de vista con ránking de productos por farmacia

In [0]:
%sql
-- Vista con ranking por farmacia
CREATE OR REPLACE VIEW farma.gold.top_products_by_pharmacy AS
SELECT
  *,
  DENSE_RANK() OVER (
    PARTITION BY id_farmacia
    ORDER BY sales_usd DESC, units_sold DESC, package_ndc_11
  ) AS rnk_by_sales_in_pharmacy
FROM farma.gold.product_lifetime_by_pharmacy;

## 8. Verificaciones

In [0]:
%sql
-- Deben existir:
SELECT COUNT(*) AS n FROM farma.gold.product_lifetime_by_pharmacy;
SELECT COUNT(*) AS n FROM farma.gold.daily_kpis_by_pharmacy_base;
SELECT COUNT(*) AS n FROM farma.gold.client_lifetime;

-- Muestras:
SELECT * FROM farma.gold.top_products_by_pharmacy ORDER BY id_farmacia DESC LIMIT 7;
SELECT * FROM farma.gold.daily_kpis_by_pharmacy_base ORDER BY order_date DESC, id_farmacia LIMIT 10;
SELECT * FROM farma.gold.top_clients_by_orders;

client_id,first_order_date,last_order_date,orders_cnt,net_sales,avg_order_value,rnk_by_orders
CLI-365,2025-09-11,2025-09-12,4,593.7099999999999,148.42749999999998,1
CLI-580,2025-09-11,2025-09-12,3,1966.94,655.6466666666666,2
CLI-462,2025-09-11,2025-09-12,3,1107.86,369.28666666666663,3
CLI-129,2025-09-11,2025-09-12,3,436.81,145.60333333333332,4
CLI-823,2025-09-11,2025-09-11,3,302.26,100.75333333333333,5
CLI-102,2025-09-11,2025-09-11,3,232.95,77.64999999999999,6
CLI-246,2025-09-11,2025-09-12,3,164.16000000000003,54.720000000000006,7
CLI-788,2025-09-11,2025-09-12,3,151.44,50.48,8
CLI-028,2025-09-12,2025-09-12,3,87.49,29.16333333333333,9
CLI-447,2025-09-11,2025-09-12,3,67.66,22.55333333333333,10
